# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Getting current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [20]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# Make a connection to a Cassandra instance of Apache Cassandra

from cassandra.cluster import Cluster
try:
    cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session

    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [6]:
# Create a Keyspace named 'event'
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS event 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# Set KEYSPACE to the keyspace specified above (event)
try:
    session.set_keyspace('event')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [8]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
'''The task here is to get the records for specific sessionId and itemsInSession. So, I made sessionId my primary key and itemsInSession as my 
Clustering column. This makes it possible to use these two fields in the where clause'''

query = "CREATE TABLE IF NOT EXISTS artist_library "
query = query + "(sessionId text, itemsInSession text, artist text, song_title text, length text, PRIMARY KEY (sessionId, itemsInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)          
    

In [9]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
##Assign the INSERT statements into the `query` variable
        query = "INSERT INTO artist_library (sessionId, itemsInSession, artist, song_title, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        ##Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (line[8],line[3],line[0],line[9],line[5])) 
        #sessionId, itemsInSession, artist, song_title, length are the 9th, 4th, 1st, 10th and 6th items in the list respectively      

In [10]:
## Add in the SELECT statement to verify the data was entered into the table. Select the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
query = "Select sessionId, itemsInSession, artist, song_title, length from artist_library where sessionId = '338' and itemsInSession = '4'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.sessionid, row.itemsinsession, row.artist, row.song_title, row.length)    

338 4 Faithless Music Matters (Mark Knight Dub) 495.3073


In [14]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

'''There is a need to run a query to pull data for specific userid and sessionid. So, I made userid my Primary key and sessionid 
the clustering column.'''

query = "CREATE TABLE IF NOT EXISTS user_library "
query = query + "(userId text, sessionId text, itemsInSession text, artist text, song_title text, firstName text, lastName text,  PRIMARY KEY ((userId, sessionId), itemsInSession)) \
WITH CLUSTERING ORDER BY (itemsInSession DESC)"
try:
    session.execute(query)
except Exception as e:
    print(e)
 

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO user_library (userId, sessionId, itemsInSession, artist, song_title, firstName, lastName)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (line[10], line[8],line[3],line[0],line[9],line[1],line[4]))
        #userId, sessionId, itemsInSession, artist, song_title, firstName, lastName are the 11th, 9th, 4th, 1st, 2nd 
        #and 5th recrds in the list respectively
        

query = "Select userId, sessionId, itemsInSession, artist, song_title, firstName, lastName from user_library where userid = '10' and sessionid = '182' "

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.userid, row.sessionid, row.itemsinsession, row.artist, row.song_title, row.firstname, row.lastname)                       

10 182 3 Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz
10 182 2 Sebastien Tellier Kilometer Sylvie Cruz
10 182 1 Three Drives Greece 2000 Sylvie Cruz
10 182 0 Down To The Bone Keep On Keepin' On Sylvie Cruz


In [18]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the 
#song 'All Hands Against His Own'

'''Since there is a need to pull data based on song title, I made song_title the primary key. I could have decided not to 
use a clustering column. However, the implication is that of multiple users listen to the same song, previously entered records
will be overwritten, leaving only the last record. In order to prevent this problem, I made firstName and lasteName the 
Clustering columns'''

query = "CREATE TABLE IF NOT EXISTS user_song_library "
query = query + "(song_title text, userId text, firstName text, lastName text,  PRIMARY KEY (song_title, userId))"
try:
    session.execute(query)
except Exception as e:
    print(e)
    
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO user_song_library (song_title, userId, firstName, lastName)"
        query = query + " VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9],line[10], line[1],line[4]))
        #song_title, firstName, lastName are the 10th, 2nd and 5th records in the list respectively
       

query = "Select song_title, userId, firstName, lastName from user_song_library where song_title='All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.song_title, row.userid, row.firstname, row.lastname)   
                    

All Hands Against His Own 29 Jacqueline Lynch
All Hands Against His Own 80 Tegan Levine
All Hands Against His Own 95 Sara Johnson


In [17]:
## Drop the table before closing out the sessions
query = "drop table artist_library"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "drop table user_library"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)    
    
query = "drop table user_song_library"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)      

Error from server: code=2200 [Invalid query] message="unconfigured table artist_library"


In [23]:
session.shutdown()
cluster.shutdown()